In [6]:
from fastai.vision.all import *
from fastai.vision.widgets import *
from bs4 import BeautifulSoup
import requests

<img src="https://github.com/pds2021/a5-LukasHoerner/releases/download/0.1/beerino.png">

Drinking beer is a challenging task on its own. On top of that, the process of choosing the beer to drink is one of the main problems to solve since the origin of human kind! Especially during the pandemic, this process got even more difficult: While one could just ask the waiter for a recommendation or just pick the most famous beer in a bar, this totally changed since bars are closed now. For this reason, beerino helps you to pick the right beers while shopping. Just upload a picture of the beer you want to buy and beerino will show you the rating from <a href="https://bierbasis.de/">bierbasis.de</a> - one of Germanys biggest websited to rate beer.

Currently beerino supports the classification of 198 different types of beer from Germany. If beerino cannot provide the right answer, you can check <a href="https://github.com/pds2021/a5-LukasHoerner/releases/download/0.1/available_categories.txt">here</a>, if the app supports your chosen beer.
As beer is the backbone of German idendity, this app will unite Germany in a way, the Solidaritätszuschlag never did.

>  *“You can't be a real country unless you have a beer and an airline - it helps if you have some kind of football team, or some nuclear weapons, but in the very least you need a beer.”* Frank Zappa




In [47]:
download_url('https://github.com/pds2021/a5-LukasHoerner/releases/download/0.1/export.pkl', 'export.pkl')
learn_inf = load_learner('export.pkl', cpu=True)
btn_upload = widgets.FileUpload()
btn_run = widgets.Button(description='Classify')
lbl_info = widgets.Label()
lbl_pred = widgets.Label()
out_pl = widgets.Output()


def rategetter(URL):
    results = requests.get(URL)
    src = results.content
    soup = BeautifulSoup(src, 'lxml')
    ratings = soup.find_all(class_="hidden-xs background-none")
    resultset = ratings[0].find_all('strong')
    nr_ratings = re.findall(r'\d+', resultset[2].contents[0])
    rating = re.findall(r'\d+\.\d+', resultset[5].contents[0])
    return [rating[0], nr_ratings[0]]

def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    pred,pred_idx,probs = learn_inf.predict(img)
    URL = 'https://bierbasis.de/bier/' + pred
    rating, nr_ratings = rategetter(URL)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'
    lbl_info.value= f'rated with: {rating}/10 on {nr_ratings} reviews;     more information on {URL}'
    with out_pl: display(img.to_thumb(512,512))

btn_run.on_click(on_click_classify)

VBox([widgets.Label('Show me your beer!'), 
      btn_upload, btn_run, lbl_pred, lbl_info, out_pl])

6.6
